In [1]:
import os
import pandas as pd

root_pth = '/home/ddon0001/PhD/experiments/scaled/no_div_constraint/'
ds_summary_path = os.path.join(root_pth, 'summary.csv')
ds_info = pd.read_csv(ds_summary_path)[['ds_name', 'det_path']]

In [2]:
from tqdm import tqdm
from _iterative_solve import load_sol_files
from _iterative_sample import populate_target_label

dfs = []
for row in tqdm(ds_info.itertuples()):
    # read all edges, solution graph, ground truth, sol_gt_mapping
    sol_pth = os.path.join(root_pth, row.ds_name)
    _, all_edges, solution_graph, gt_graph, _, sol_to_gt = load_sol_files(sol_pth)
    # assign "edge_in_gt" and "is_correct" to all real edges
    populate_target_label(all_edges, solution_graph, gt_graph, sol_to_gt)
    all_edges['solution_correct'] = (all_edges.flow > 0) == (all_edges.oracle_is_correct)
    all_edges['solution_incorrect'] = all_edges.solution_correct == False
    all_edges['ds_name'] = row.ds_name
    dfs.append(all_edges[(all_edges.u >= 0) & (all_edges.v >= 0)])
overall_df = pd.concat(dfs)
overall_df.to_csv(os.path.join(root_pth, 'all_edges_with_target.csv'), index=False)

26it [00:50,  1.95s/it]


In [3]:
plot_order = overall_df[overall_df.solution_incorrect].ds_name.value_counts().index

In [ ]:
# plot sensitivity diff of is_correct edges vs. not is correct edges, split by flow >0 and flow =0 
import seaborn as sns

migration_only = overall_df[(overall_df.u >= 0) & (overall_df.v >= 0)]
migration_only['in_solution'] = migration_only.flow > 0
grid = sns.FacetGrid(migration_only, height=6, aspect=0.7, col='ds_name', col_wrap=4, sharex=True, sharey=False, col_order=plot_order)
grid.map_dataframe(sns.violinplot, x='solution_correct', y='sensitivity_diff', inner='quart', hue='in_solution', palette='deep', split=True, gap=0.05, cut=0)

grid.set_titles(size=16)
grid.set_axis_labels(size=14)
grid.set_xticklabels(size=12)
grid.set_yticklabels(size=12)
grid.add_legend()

In [ ]:
# on average, how many incorrect edges can we find in that initial sample from **solution** edges
# by using the sensitivity_diff as a weight?
# we need sensitivity diff to be small in order for us to sample

# scaling sensitivity diff

# we first derive a scaled column - sensitivity_diff scaled to [0, 1] range
# then, we derive 1-scaled_sensitivity_diff. This means the smaller the initial diff, the more likely we are to sample it
# then for each dataset
# we take 30 samples of size 30:
# - just pure random sample
# - weighted by distance
# - weighted by inverse sensitivity diff thing
# and store the number of incorrect solution edges sampled, for each sampling type

# we also generate solutions disallowing merges and do the same thing?

In [41]:
fluo_edges = overall_df[overall_df.ds_name == 'Fluo-N2DL-HeLa_02']
len(fluo_edges)

250150

In [50]:
fluo_in_sol = fluo_edges[(fluo_edges.flow > 0) & (fluo_edges.u >= 0) & (fluo_edges.v >= 0)]
fluo_in_sol.sort_values(by='distance', ascending=False)[:100]['solution_incorrect'].sum()


74

In [52]:
fluo_in_sol.sort_values(by='sensitivity_diff', ascending=True)[101:200]['solution_incorrect'].sum()


21

In [53]:
fluo_in_sol.sort_values(by='distance', ascending=False)[:200]['solution_incorrect'].sum()


105